In [ ]:
import numpy as np
import math
import numpy.matlib as matlib
import operator
from time import perf_counter
import scipy

Mount drive to Colab:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Nativgate to the right directory to save MCMC data:

In [ ]:
%cd /content/drive/My\ Drive/msc_dissertation/experiment_1

/content/drive/My Drive/msc_dissertation/experiment_1


# BPS MCMC and Gaussian distributions

The following gives the basic BPS algorithm. The outputs of the function are the turning positions of the particle, the time instants at those turning, the velocities, the computational time at those turning, and the total number of pdf evaluations at those turnings.

BPS algorithm:

In [ ]:
def BPS_basic(x0, v0, Time, lambda_ref, prob_dist):
    
    cpu_time_executed = perf_counter()
    computational_times = [0]
    start_time = cpu_time_executed
    
    turn_pts = [x0]
    list_of_velo = [v0]
    striding_times = [0]
    total_evals_list = [0]
    dim = x0.size
    i = 1
    x = x0
    v = v0
    t = 0
    total_evals = 0

    
    
    while t < Time:

        tau_bounce, num_evals = prob_dist.bounce_time(x, v)
        beta = 1/lambda_ref
        tau_ref = np.random.exponential(scale = beta)
        tau = min(tau_bounce, tau_ref)
        
        x = x + tau*v
        t = t + tau
        
        if tau_ref < tau_bounce:
            v = np.random.standard_normal(dim)
        else:
            v = v - 2*(np.sum(prob_dist.ener_grad(x)*v))/(np.sum(prob_dist.ener_grad(x)*
                                                                 prob_dist.ener_grad(x)))*prob_dist.ener_grad(x)
            num_evals = num_evals + 1
        
        
        cpu_time_executed = perf_counter() - start_time
        total_evals = total_evals + num_evals
        
        
        list_of_velo.append(v)
        turn_pts.append(x)
        striding_times.append(t)
        computational_times.append(cpu_time_executed)
        total_evals_list.append(total_evals)
        
        
        
        i = i+1

    return turn_pts, list_of_velo, striding_times, total_evals_list, computational_times    
    # x_list, v_list, t_list, evals_list, cpu_time_list

Multi-variate Gaussian distribution implemented as a class:

In [ ]:
class Gaussian:
    def __init__(self, mean, cov_mat):
        self.mean = mean
        self.cov_mat = cov_mat
        self.dim = mean.size
        
        
    def energy(self, x):   # constant coeff is discarded
        return 0.5*((x - self.mean).dot(np.linalg.inv(self.cov_mat)).dot(x - self.mean))      
    
    
    def ener_grad(self, x):
        return (np.linalg.inv(self.cov_mat)).dot(x - self.mean)

    
    def bounce_time(self, x, v):
        V = np.random.uniform()*(-1)+1   # V has uniform distribution over (0,1]. The issue of math.log(0) avoided.
        
        sigma_inv_x_minus_mu = (np.linalg.inv(self.cov_mat)).dot(x - self.mean)
        sigma_inv_v = (np.linalg.inv(self.cov_mat)).dot(v)
        abbr = np.sum(sigma_inv_v*v)
        thres = np.sum(sigma_inv_x_minus_mu*v)
        
        if thres > 0:
            tau_bounce = (1/abbr)*((math.sqrt(thres**2 - 2*abbr*math.log(V))) - thres)
        else:
            tau_bounce = (1/abbr)*(math.sqrt(-2*abbr*math.log(V)) - thres)
        
        return tau_bounce, 2

The following function allows us to compute the particle's position and velocity at any time during its travel:

In [ ]:
def x_v_t_arbitrary_times(turn_pts, list_of_velo, striding_times, intermediate_times):
    
    # turn_pts, list_of_velo, striding_times, intermediate_times can be list or np.array

    num_changes = len(turn_pts)
    num_required_times = len(intermediate_times)
    tiling_interm_times = np.transpose(np.tile(intermediate_times, (num_changes, 1)))
    testing_mat_1 = tiling_interm_times - np.tile(striding_times, (num_required_times, 1))
    testing_mat_2 = np.where(testing_mat_1 >= 0, 1, 0)
    indices_no_later_than = np.sum(testing_mat_2, axis = 1) - 1
        
    turn_pts_no_later = [turn_pts[i] for i in indices_no_later_than]
    velo_no_later = [list_of_velo[i] for i in indices_no_later_than]
    stride_time_no_later = [striding_times[i] for i in indices_no_later_than]
    
       
    interm_times = list(intermediate_times)
        
    list_of_coasting_times = list(map(operator.sub, interm_times, stride_time_no_later))
    distance_strided = list(map(operator.mul, list_of_coasting_times, velo_no_later))
    locations_at_required_times = list(map(operator.add, turn_pts_no_later, distance_strided))
        

    return locations_at_required_times, velo_no_later, interm_times